In [3]:
# import library
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import os
import plotly.express as px
import geopandas as gpd
import folium
import branca
from tqdm.notebook import tqdm
from folium import Marker, FeatureGroup
from folium.plugins import MarkerCluster, FeatureGroupSubGroup, Search

In [5]:
# get the current working directory
cwd = os.getcwd()

# print the current working directory
print(f"Current working directory: {cwd}")

path = '/Users/johnn/Documents/Research/codebook/covid/'
os.chdir(path)

# get the current working directory
cwd = os.getcwd()
# print the current working directory
print(f"Current working directory: {cwd}")


Current working directory: c:\Users\johnn\Documents\Research\codebook\covid
Current working directory: c:\Users\johnn\Documents\Research\codebook\covid


In [6]:
# define global parameters for web scrapping.
headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36'}

In [11]:
def get_link(page):
    df_link = pd.DataFrame()
    link = []
    title = []
    url = 'https://ss.shanghai.gov.cn/service/wsjkw/search'
    key = '新增本土新冠肺炎'
    params = {"q": key, 'all':1, 'page': page, 'siteId':'wsjkw.sh.gov.cn', 'siteArea':'all'}
    results = requests.get(url, params=params, timeout=30, headers=headers)
    results.encoding = 'utf-8'
    soup = BeautifulSoup(results.text, 'lxml')
    for x in soup.find_all(class_ = 'result'):
        title.append(x.find('a').get('title'))
        link.append(x.find(class_='url').find('a').get('href'))

    df_link['link'] = link
    df_link['title'] = title

    return df_link

In [393]:
df_link = pd.DataFrame()
num_pages = 4
for page in tqdm(range(num_pages)):
    df_link = df_link.append(get_link(page+1), ignore_index=True)

# clean date
date = []
for idx, row in tqdm(df_link.iterrows(), total=df_link.shape[0]):
    date_ = re.findall('(?:\d+年)?(\d+月\d+日)', row['title'])    
    if date_:
        date.append('2022年' + date_[0])
    else:
        date.append('2022年3月28日')

df_link['date'] = pd.to_datetime(date, format='%Y年%m月%d日')
df_link.sort_values(by='date', inplace=True)
df_link.reset_index(inplace=True,drop=True)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

In [363]:
def parse_report(link):
    # define regex
    regex = '(?:病例|无症状感染者)(\d+)(?:\、|\—)?(?:病例|无症状感染者)?(\d*)' 
    
    r = requests.get(link, headers=headers)
    r.encoding = 'utf-8'
    soup = BeautifulSoup(r.text, 'lxml')
    # find article content
    article = soup.find(class_="Article_content")
    
    # create flags
    need_date_flag = True
    previous_context = ''

    # different case types
    case_types = ['风险人群筛查中', 
                  '本市闭环隔离管控人员', 
                  '此前报告的本土无症状感染者']
    # different positive case types
    positive_case_types = ['病例', '无症状感染者']
    # create empty df
    df = pd.DataFrame()
    for child in article.children:
        # find date
        if need_date_flag:
            capture_date = re.search('\d+年\d+月\d+日', child.text)
            if capture_date:
                date = pd.to_datetime(capture_date.group(0), format="%Y年%m月%d日")
                need_date_flag = False
                continue
        
        # find the content
        need_continue_flag = not bool(re.search("[\。\）]$", child.text))
        article_flag = '居住于' in child.text
        if article_flag & need_continue_flag:
            previous_context += child.text
            continue
        elif article_flag & ~need_continue_flag:
            context = previous_context + child.text
            district_ = re.findall('(?:居住于[\，|\"]?)?([\u4E00-\u9FA5]+区|来沪求职)(?:[\u4E00-\u9FA5\da-zA-Z]*)(?=\，|\s)', context)
            #district_ = re.findall("(?:居住于)(?:\，)?([\u4E00-\u9FA5]+)(?=\，)", context)
            cases_ = re.findall(regex, context)
            try:
                assert len(cases_) == len(district_)
            except:
                print("Different length of case number and districts")
                print(context)
                print('date: {}'.format(date.strftime(format='%Y-%m-%d')))
                print("The length of case: {}".format(len(cases_)))
                print("The length of district: {}".format((district_)))
            cases_number = [int(case[1]) - int(case[0]) + 1 if case[1] else 1 for case in cases_]
            
            # define the case type
            if case_types[0] in context:
                case_type_ = 'A'
            elif case_types[1] in context:
                case_type_ = 'B'
            elif case_types[2] in context:
                case_type_ = 'C'
            else:
                case_type_ = None
            
            # define the positive case type
            if positive_case_types[0] in context:
                positive_case_type_ = 'positive'
            elif positive_case_types[1] in context:
                positive_case_type_ = 'asym'
            else:
                positive_case_type_ = None 

            # reset previous context
            previous_context = ''

            df_ = pd.DataFrame({'cases':cases_, 
                                'district':district_, 
                                'cases_number': cases_number})
            df_['case_type'] = case_type_
            df_['positive_case_type'] = positive_case_type_
        else:
            continue 
        
        # append the dataframe
        df = df.append(df_, ignore_index=True)
        df['date'] = date
    
    return df

    

In [397]:
df_link.loc[df_link['date'] > pd.to_datetime('2022-03-15'), :]

,link,title,date
39,http://wsjkw.sh.gov.cn/xwfb/20220317/46f73d4d7...,上海2022年3月16日，新增本土新冠肺炎确诊病例8例 新增本土无症状感染者150例 新增境...,2022-03-16
40,http://wsjkw.sh.gov.cn/xwfb/20220318/dc6e1aca6...,上海2022年3月17日，新增本土新冠肺炎确诊病例57例 新增本土无症状感染者203例 新增...,2022-03-17
41,http://wsjkw.sh.gov.cn/xwfb/20220319/08e4bb662...,上海2022年3月18日，新增本土新冠肺炎确诊病例8例 新增本土无症状感染者366例 新增境...,2022-03-18
42,http://wsjkw.sh.gov.cn/xwfb/20220320/309cf8bf0...,上海2022年3月19日，新增本土新冠肺炎确诊病例17例 新增本土无症状感染者492例 新增...,2022-03-19
43,http://wsjkw.sh.gov.cn/xwfb/20220321/edd50169c...,上海2022年3月20日，新增本土新冠肺炎确诊病例24例 新增本土无症状感染者734例 新增...,2022-03-20
44,http://wsjkw.sh.gov.cn/xwfb/20220322/92cb33af7...,上海2022年3月21日，新增本土新冠肺炎确诊病例31例 新增本土无症状感染者865例 新增...,2022-03-21
45,http://wsjkw.sh.gov.cn/xwfb/20220323/560de9823...,上海2022年3月22日，新增本土新冠肺炎确诊病例4例 新增本土无症状感染者977例 新增境...,2022-03-22
46,http://wsjkw.sh.gov.cn/xwfb/20220324/197cda641...,上海2022年3月23日，新增本土新冠肺炎确诊病例4例 新增本土无症状感染者979例 新增境...,2022-03-23
47,http://wsjkw.sh.gov.cn/xwfb/20220325/f2f2ffe5d...,上海2022年3月24日，新增本土新冠肺炎确诊病例29例 新增本土无症状感染者1580例 新...,2022-03-24
48,http://wsjkw.sh.gov.cn/xwfb/20220326/210fddd35...,上海2022年3月25日，新增本土新冠肺炎确诊病例38例 新增本土无症状感染者2231例 新...,2022-03-25


In [405]:
mask = df_link['date'] > pd.to_datetime('2022-03-13')
df_link_subset = df_link.loc[mask, :]
df = pd.DataFrame()
for idx, row in tqdm(df_link_subset.iterrows(), total=df_link_subset.shape[0]):
    df = df.append(parse_report(row['link']), ignore_index=True)

  0%|          | 0/43 [00:00<?, ?it/s]

In [403]:
df

,cases,district,cases_number,case_type,positive_case_type,date
0,"(1, )",黄浦区,1,A,positive,2022-03-14
1,"(2, )",闵行区,1,A,positive,2022-03-14
2,"(3, )",松江区,1,A,positive,2022-03-14
3,"(4, )",浦东新区,1,A,positive,2022-03-14
4,"(5, )",徐汇区,1,None,positive,2022-03-14
...,...,...,...,...,...,...
11103,"(15199, 15264)",杨浦区,66,A,asym,2022-04-25
11104,"(15265, 15270)",闵行区,6,A,asym,2022-04-25
11105,"(15271, 15294)",宝山区,24,A,asym,2022-04-25
11106,"(15295, 15314)",嘉定区,20,A,asym,2022-04-25


In [349]:
test = re.findall("(?:居住于[\，|\"]?)?([\u4E00-\u9FA5]+区[\u4E00-\u9FA5\da-zA-Z\s]*|来沪求职)(?=\，|\s)",'无症状感染者35，男，11岁，居住于浦东新区妙境路124弄，无症状感染者36，男，36岁，居住于嘉定区梅园路300弄，无症状感染者37，男，47岁，居住于嘉定区张掖路355号，无症状感染者38，女，55岁，居住于嘉定区中大街128号，无症状感染者39，男，41岁，居住于松江区通波路725弄，无症状感染者40，女，43岁，居住于松江区新松江路259弄，无症状感染者41，女，25岁，居住于松江区新飞路1999弄，无症状感染者42，女，19岁，居住于松江区新飞路1999弄，无症状感染者43，女，18岁，居住于松江区龙源路555号，无症状感染者44，男，25岁，居住于松江区王家厍路55弄，无症状感染者45，男，50岁，居住于静安区茂名北路93号，无症状感染者46，男，55岁，居住于金山区学府路2208弄，无症状感染者47，男，69岁，居住于静安区沪太路1329弄，无症状感染者48，女，39岁，居住于嘉定区崇教路267号对面工地宿舍，无症状感染者49，女，39岁，居住于虹口区逸仙路288弄，无症状感染者50，男，28岁，居住于徐汇区中山西路1788弄，无症状感染者51，男，69岁，居住于静安区沪太路1170弄，无症状感染者52，女，26岁，居住于嘉定区封周路885弄，无症状感染者53，女，58岁，居住于嘉定区南新路317号，无症状感染者54，女，46岁，居住于徐汇区华发路406弄，无症状感染者55，男，60岁，居住于黄浦区西藏南路1129弄，无症状感染者56，男，19岁，居住于闵行区东川路800号，无症状感染者57，女，23岁，居住于闵行区东川路800号，无症状感染者58，男，21岁，居住于闵行区东川路800号，无症状感染者59，男，23岁，居住于闵行区东川路800号，无症状感染者60，女，23岁，居住于闵行区东川路800号，无症状感染者61，男，18岁，居住于闵行区东川路800号，无症状感染者62，男，20岁，居住于闵行区东川路800号，无症状感染者63，男，19岁，居住于闵行区东川路800号，无症状感染者64，男，59岁，居住于闵行区剑川路中铁四局工地宿舍，无症状感染者65，女，17岁，居住于闵行区东川路800号，无症状感染者66，男，58岁，居住于闵行区剑川路中铁四局工地宿舍，无症状感染者67，男，28岁，居住于闵行区剑川路中铁四局工地宿舍，无症状感染者68，男，23岁，居住于闵行区东川路800号，无症状感染者69，女，35岁，居住于闵行区安乐村，无症状感染者70，女，47岁，居住于徐汇区上中西路151弄，无症状感染者71，男，34岁，居住于松江区泗宝路458弄，无症状感染者72，女，70岁，居住于徐汇区中山南二路945弄，无症状感染者73，女，44岁，居住于徐汇区日晖六村，无症状感染者74，男，50岁，居住于徐汇区龙水南路385弄，无症状感染者75，女，25岁，来沪求职，无症状感染者76，女，41岁，居住于嘉定区汇旺东路1295弄，无症状感染者77，女，33岁，居住于嘉定区南新路626弄，无症状感染者78，女，51岁，居住于嘉定区小东街，无症状感染者79，男，57岁，居住于嘉定区红石路699弄，无症状感染者80，女，54岁，居住于嘉定区小东街，无症状感染者81，女，51岁，居住于徐汇区凯滨路19弄，无症状感染者82，女，9岁，居住于宝山区场北路399弄，无症状感染者83，男，11岁，居住于徐汇区龙吴路1343弄，无症状感染者84，男，33岁，居住于金山区海盛路500弄，无症状感染者85，女，67岁，居住于黄浦区顺昌路612弄，无症状感染者86，女，61岁，居住于黄浦区顺昌路612弄，无症状感染者87，女，56岁，居住于黄浦区顺昌路612弄，无症状感染者88，女，84岁，居住于黄浦区顺昌路612弄，无症状感染者89，男，62岁，居住于黄浦区顺昌路612弄，无症状感染者90，男，69岁，居住于黄浦区顺昌路612弄，无症状感染者91，女，62岁，居住于黄浦区顺昌路612弄，无症状感染者92，男，74岁，居住于黄浦区顺昌路580号，无症状感染者93，女，60岁，居住于黄浦区顺昌路580号，无症状感染者94，男，64岁，居住于黄浦区顺昌路612弄，无症状感染者95，男，89岁，居住于黄浦区顺昌路612弄，无症状感染者96，男，66岁，居住于黄浦区顺昌路612弄，无症状感染者97，男，61岁，居住于黄浦区顺昌路598号，无症状感染者98，男，68岁，居住于黄浦区顺昌路612弄，无症状感染者99，女，69岁，居住于黄浦区顺昌路612弄，无症状感染者100，女，48岁，居住于黄浦区大沽路183弄，无症状感染者101，女，50岁，居住于嘉定区三里村，无症状感染者102，女，8岁，居住于静安区西藏北路970弄，无症状感染者103，女，15岁，居住于闵行区都会路3552号，无症状感染者104，男，39岁，居住于闵行区虹梅南路1781弄，无症状感染者105，女，16岁，居住于松江区仓汇路401弄，无症状感染者106，男，56岁，居住于嘉定区崇教路267号对面工地宿舍，无症状感染者107，男，50岁，居住于嘉定区崇教路267号对面工地宿舍，无症状感染者108，男，35岁，居住于嘉定区崇教路267号对面工地宿舍，无症状感染者109，男，26岁，居住于嘉定区崇教路267号对面工地宿舍，无症状感染者110，男，32岁，居住于静安区临汾路894弄，无症状感染者111，女，60岁，居住于金山区建农村，均系本市报告本土确诊病例或无症状感染者的密切接触者，即被隔离管控，其间新冠病毒核酸检测结果异常，经市疾控中心复核结果为阳性。经市级专家会诊，综合流行病学史、临床症状、实验室检测和影像学检查结果等，诊断为无症状感染者。')

In [351]:
a=['浦东新区', '嘉定区', '嘉定区', '嘉定区', '松江区', '松江区', '松江区', '松江区', '松江区', '松江区', '静安区', '金山区', '静安区', '嘉定区', '虹口区', '徐汇区', '静安区', '嘉定区', '嘉定区', '徐汇区', '黄浦区', '闵行区', '闵行区', '闵行区', '闵行区', '闵行区', '闵行区', '闵行区', '闵行区', '闵行区', '闵行区', '闵行区', '闵行区', '闵行区', '闵行区', '徐汇区', '松江区', '徐汇区', '徐汇区', '徐汇区', '嘉定区', '嘉定区', '嘉定区', '嘉定区', '嘉定区', '徐汇区', '宝山区', '徐汇区', '金山区', '黄浦区', '黄浦区', '黄浦区', '黄浦区', '黄浦区', '黄浦区', '黄浦区', '黄浦区', '黄浦区', '黄浦区', '黄浦区', '黄浦区', '黄浦区', '黄浦区', '黄浦区', '黄浦区', '嘉定区', '静安区', '闵行区', '闵行区', '松江区', '嘉定区', '嘉定区', '嘉定区', '嘉定区', '静安区', '金山区']

In [354]:
'来沪求职' in a

False

In [317]:
170-102+1

69